In [1]:
!pip install spacy==2.3.2
# print(torch.cuda.is_available())

     |████████████████████████████████| 10.0MB 4.0MB/s 
     |████████████████████████████████| 2.1MB 55.3MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext import data
from torchtext.data import Field
from torchtext.data import BucketIterator


import spacy
import numpy as np
import os
import random
import pandas as pd
import time
import math

from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# !pip freeze > requirements.txt
# !cat requirements.txt | grep spacy
# !cat requirements.txt | grep torch
# !cat requirements.txt | grep torchtext
# !cat requirements.txt | grep torchvision

In [5]:
!pip install datasets

     |████████████████████████████████| 153kB 5.7MB/s 
     |████████████████████████████████| 245kB 36.0MB/s 
     |████████████████████████████████| 17.7MB 215kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [6]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric
dataset = load_dataset("cnn_dailymail", '3.0.0')
train=dataset['train']
test=dataset['test']
validate=dataset['validation']

print(len(test))

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.
11490


In [7]:
type(train)

datasets.arrow_dataset.Dataset

In [8]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [9]:
stories_train = []
summaries_train = []

stories_test = []
summaries_test = []

stories_validate = []
summaries_validate = []

num = 100000000
for i in range(len(train)):
    story = train[i]['article']
    summary = train[i]['highlights']
    stories_train.append(story)
    summaries_train.append(summary)
    if i == num:
       break
for i in range(len(test)):
    story = test[i]['article']
    summary = test[i]['highlights']
    stories_test.append(story)
    summaries_test.append(summary)
    if i == num:
       break
for i in range(len(validate)):
    story = validate[i]['article']
    summary = validate[i]['highlights']
    stories_validate.append(story)
    summaries_validate.append(summary)
    if i == num:
       break

del train
del test
del validate
del dataset

In [10]:
DATA_TRAIN = {"story": [story for story in stories_train], "summary": [summary for summary in summaries_train]}
DATA_TEST = {"story": [story for story in stories_test], "summary": [summary for summary in summaries_test]}
DATA_VALIDATE = {"story": [story for story in stories_validate], "summary": [summary for summary in summaries_validate]}

stories_train.clear()
summaries_train.clear()
stories_test.clear()
summaries_test.clear()
stories_validate.clear()
summaries_validate.clear()


In [11]:
df_train = pd.DataFrame(DATA_TRAIN, columns=["story", "summary"])
df_test = pd.DataFrame(DATA_TEST, columns=["story", "summary"])
df_validate = pd.DataFrame(DATA_VALIDATE, columns=["story", "summary"])

DATA_TRAIN.clear()
DATA_TEST.clear()
DATA_VALIDATE.clear()

df_train.to_csv("train_data.csv")
df_test.to_csv("test_data.csv")
df_validate.to_csv("validation_data.csv")

In [12]:
# df_train.to_json("train_data.json", orient="records", lines=True)
# df_test.to_json("test_data.json", orient="records", lines=True)
# df_validate.to_json("validation_data.json", orient="records", lines=True)

# del df_train
# del df_test
# del df_validate

In [13]:
!python -m spacy download en
spacy_en = spacy.load('en')
MAX_LENGTH = 400

def tokenize(text):
    tokens = spacy_en.tokenizer(text)
    length = len(tokens)
    
    cnt = 0
    toRet = []
    
    for tok in tokens:
        if cnt < length and cnt < MAX_LENGTH:
            toRet.append(tok.text)
            cnt += 1
        
        else:
            break
    
    return toRet


     |████████████████████████████████| 12.1MB 1.1MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=bc5551f87e37b5abd94683b50b23cc1016b16235b3dd726ab5c08f0458b25591
  Stored in directory: /tmp/pip-ephem-wheel-cache-af0zr_tj/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [14]:
story_field = Field(tokenize = tokenize, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
summary_field = Field(tokenize =tokenize, init_token = '<sos>', eos_token = '<eos>', lower = True)
fields = {'story': ('story', story_field), 'summary': ('summary', summary_field)}



In [16]:
train_data, validation_data, test_data = data.TabularDataset.splits(path = '',
                                        train = 'train_data.csv',
                                        validation = 'validation_data.csv',
                                        test = 'test_data.csv',
                                        format = 'csv',
                                        fields = fields)

In [17]:
BATCH_SIZE = 8

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
     batch_size = BATCH_SIZE,
     shuffle = True,
     sort_within_batch = True,
     sort_key = lambda x : len(x.story),
     repeat = False,
     device = device)

In [18]:
story_field.build_vocab(train_data, min_freq = 2)
summary_field.build_vocab(train_data, min_freq = 2)

# import pickle

# with open('summary_field.pkl', 'rb') as f:
#     summary_field.vocab = pickle.load(f)

# with open('story_field.pkl', 'rb') as f:
#     story_field.vocab = pickle.load(f)

print(len(story_field.vocab))
print(len(summary_field.vocab))

300
34


In [19]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

save_vocab(story_field.vocab, "story_field.pkl")
save_vocab(summary_field.vocab, "summary_field.pkl")

In [20]:
# del train_data
# !rm train_data.json
# !rm test_data.json
# !rm validation_data.json
# del validation_data
# print(type(train_data))

In [21]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [22]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [27]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)  
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
                   
        input = input.unsqueeze(0)  
        embedded = self.dropout(self.embedding(input))  
        a = self.attention(hidden, encoder_outputs, mask)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedded, weighted), dim = 2)         
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [29]:
INPUT_DIM = len(story_field.vocab)
OUTPUT_DIM = len(summary_field.vocab)
ENC_EMB_DIM = 100
DEC_EMB_DIM = 100
ENC_HID_DIM = 100
DEC_HID_DIM = 100
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = story_field.vocab.stoi[story_field.pad_token]

In [30]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM).to(device)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT).to(device)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn).to(device)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [31]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = summary_field.vocab.stoi[summary_field.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
# load_checkpoint(torch.load("./Mseq2seq_attention_padding-model.pt"), model, optimizer)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.story
            trg = batch.summary

            output = model(src.to(device), src_len, trg.to(device), 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output.to(device), trg.to(device))

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
model.load_state_dict(torch.load('gloveseq2seq.pt'))


<All keys matched successfully>

In [ ]:
import torch
print(torch.cuda.is_available())
# test_loss = evaluate(model, test_iterator, criterion)

# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

True


In [ ]:
def make_summary(sentence, src_field, trg_field, model, device, max_len = 100):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [ ]:
example_idx = 2

src = vars(train_data.examples[example_idx])['story']
trg = vars(train_data.examples[example_idx])['summary']

print(f'src = {src}')
print(f'trg = {trg}')

summary, attention = make_summary(src, story_field, summary_field, model, device)

print(f'predicted trg = {summary}')

NameError: ignored

In [ ]:
example_idx = 500

src = vars(validation_data.examples[example_idx])['story']
trg = vars(validation_data.examples[example_idx])['summary']

print(f'src = {" ".join(src)}\n')
print(f'trg = {" ".join(trg)}\n')

summary, attention = make_summary(src, story_field, story_field, model, device)
summary = (" ".join(summary)).replace("\n", " ")

print(f'predicted trg = {summary}')

src = jerusalem ( cnn)after a bitter campaign , israeli prime minister benjamin netanyahu is claiming victory as his main rival isaac herzog concedes . many questions linger about exactly what this means for u.s.-israeli relations and israel 's standing in the middle east . cnn global affairs correspondent elise labott has been covering the israel election in depth and took to facebook to answer your questions about the election . here are five highlights from the chat : . 1 ) how did the race go from exit polls saying it was a neck - and - neck race to netanyahu claiming victory within a matter of hours ? i know , right ? it 's a great mystery as to how the exit polls got it so wrong . there 's one pollster here in israel named avi degani who predicted that netanyahu would win all along . he said that the polls here in israel rely too heavily on modern methodology and are n't really suitable for people outside city centers like tel aviv . there 's also the question of whether netanyah

KeyError: ignored

In [ ]:
example_idx = 10000

src = vars(validation_data.examples[example_idx])['story']
trg = vars(validation_data.examples[example_idx])['summary']

print(f'src = {" ".join(src)}\n')
print(f'trg = {" ".join(trg)}\n')

summary, attention = make_summary(src, story_field, summary_field, model, device)
summary = (" ".join(summary)).replace("\n", " ")

print(f'predicted trg = {summary}')

src = convicted boyfriend killer jodi arias is spending several hours a day video - chatting to fans from her prison cell . the murderer , who narrowly escaped the death penalty for the brutal slaying of travis alexander in 2008 , is said to be the most popular inmate getting video calls - many from girls as young as 15 . photos have surfaced showing arias pulling faces and showing off a hershey 's from the confines of the cell where she spends 23-hours - a - day . scroll down for video . all smiles : ' fans ' take snaps of murderer arias as she shows off a hershey 's bar and smiles to the camera during video chats from prison cell . last saturday alone , arias took 27 video calls in jail with pictures from some of those calls having leaked . with each call lasting up to 20 minutes , it means arias spent up to nine hours that day skyping with followers and fans from a laptop . poser : the 34-year - old killer was snapped by admirers who have been skyping with arias in her prison cell f

In [ ]:
example_idx = 999

src = vars(validation_data.examples[example_idx])['story']
trg = vars(validation_data.examples[example_idx])['summary']

print(f'src = {" ".join(src)}\n')
print(f'trg = {" ".join(trg)}\n')

summary, attention = make_summary(src, story_field, summary_field, model, device)
summary = (" ".join(summary)).replace("\n", " ")

print(f'predicted trg = {summary}')

# Factually incorrect summary. Pointer-Generator should hopefully not repeat this mistake.

src = ( cnn)it was always going to take an extraordinary performance from a very talented swimmer to beat michael phelps at his favorite event in an olympic final . for more than a decade , the great american had vanquished all comers in the 200 meters butterfly on both the world and olympic stage , but then along came chad le clos . when the south african chased down phelps in the final 50 meters of the race to snatch a dramatic fingertip victory by 0.05 seconds he created one of the defining moments of the 2012 london olympics . as a 12-year - old , le clos had gazed into his television set at home in durban watching in awe as phelps , widely regarded as the greatest swimmer of all - time , won six olympic gold medals at athens in 2004 . so perhaps it 's not surprising that lining up alongside his hero in an olympic final eight years later took on a slightly surreal air . " my dream was always to swim like michael phelps so when i raced against him in the final it was actually a craz

In [ ]:
example_idx = 124

src = vars(validation_data.examples[example_idx])['story']
trg = vars(validation_data.examples[example_idx])['summary']

print(f'src = {" ".join(src)}\n')
print(f'trg = {" ".join(trg)}\n')

summary, attention = make_summary(src, story_field, summary_field, model, device)
summary = (" ".join(summary)).replace("\n", " ")

print(f'predicted trg = {summary}')

src = ( cnn)former manchester united star danny welbeck returned to haunt his old side to send holders arsenal into the semifinals of the fa cup with a 2 - 1 victory at old trafford monday . the england striker was surplus to requirements at louis van gaal 's united , but was quick to capitalize on a mistake by antonio valencia to grab the winner just after the hour mark . a disappointing night for united was compounded by a late red card for argentina international angel di maria , who was given his marching orders after manhandling referee michael oliver . di maria was initially booked for simulation and took his protests too far by grabbing the official , which led to his instant dismissal . with 10 men , united was never able to exert any real pressure on arsenal and it took two fine saves david de gea to deny further goals from santi cazorla and alexis sanchez for the away side . victory for arsenal booked a return to wembley for a last four clash with either reading or bradford c

In [ ]:
example_idx = 12

src = vars(test_data.examples[example_idx])['story']
trg = vars(test_data.examples[example_idx])['summary']

print(f'src = {" ".join(src)}\n')
print(f'trg = {" ".join(trg)}\n')

summary, attention = make_summary(src, story_field, summary_field, model, device)
summary = (" ".join(summary)).replace("\n", " ")

print(f'predicted trg = {summary}')

src = ( cnn)more than 500 houthi rebels have been killed since the start of saudi - led military operations against yemeni shia fighters , a saudi defense ministry official said saturday , according to the state - run saudi press agency . a saudi general said saturday the nine - nation coalition has undertaken 1,200 airstrikes since they began on march 26 . gen. ahmed asiri added that the raids aim to keep the rebels from moving toward southern yemen ,   according to the spa . clashes took place friday near the saudi - yemeni border , in the najran region . saudi forces responded to mortar rounds fired by houthis on a saudi border site . three saudi military officers were killed and two others were wounded in the shelling , a defense official said , according to spa . a saudi source also confirmed to cnn 's nic robertson that three saudi soldiers were killed in the shelling . the yemeni health ministry on saturday said 385 civilians have been killed and 342 others have been wounded . t

In [ ]:
def make_html_safe(s):
  s.replace("<", "&lt;")
  s.replace(">", "&gt;")
  return s

In [ ]:
!rm -rf predicted
!rm -rf target
!mkdir predicted
!mkdir target

In [ ]:
def store_data(data, src_field, trg_field, model, device, max_len = 100):
    
    trgs = []
    pred_trgs = []
    idx = 1
    cnt = 0
    for datum in data:
        cnt += 1
        if cnt == 100:
          break
        src = vars(datum)['story']
        trg = vars(datum)['summary']
        
        src = ' '.join(src)
        trg = ' '.join(trg)
        pred_trg, _ = make_summary(src, src_field, trg_field, model, device, max_len)
        pred_trg = pred_trg[:-1]
        pred_trg = ' '.join(pred_trg)

        trg = trg.replace("\n", "")
        pred_trg = pred_trg.replace("\n", "")
        trg = trg.split(".")
        pred_trg = pred_trg.split(".")

        trg = "\n".join(trg)
        pred_trg = "\n".join(pred_trg)
        trg = make_html_safe(trg)
        pred_trg = make_html_safe(pred_trg)

        pred_file = open("./predicted/file." + str(idx) + ".txt", "w+")
        trg_file = open("./target/file." + str(idx) +".txt", "w+")

        pred_file.write(pred_trg)
        trg_file.write(trg)
        pred_file.close()
        trg_file.close()
        idx += 1
        #cut off <eos> token
        # 
        
        # pred_trgs.append(pred_trg)
        # trgs.append([trg])
        
    # return bleu_score(pred_trgs, trgs)

In [ ]:
# !mkdir predicted
# !mkdir targety
store_data(test_data, story_field, summary_field, model, device)

In [ ]:
!cp -r target "./drive/My Drive/IRE"
!cp -r predicted "./drive/My Drive/IRE"
# !pip install pyrouge
# !ls target

In [ ]:
from pyrouge import Rouge155

r = Rouge155('pythonrouge/pythonrouge/RELEASE-1.5.5')
r.system_dir = './predicted/'
r.model_dir = './target'
r.system_filename_pattern = 'file.(\d+).txt'
r.model_filename_pattern = 'file.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2020-10-28 06:26:11,064 [MainThread  ] [INFO ]  Set ROUGE home directory to pythonrouge/pythonrouge/RELEASE-1.5.5.
2020-10-28 06:26:11,065 [MainThread  ] [INFO ]  Writing summaries.
2020-10-28 06:26:11,067 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp8rlkuve6/system and model files to /tmp/tmp8rlkuve6/model.
2020-10-28 06:26:11,068 [MainThread  ] [INFO ]  Processing files in ./predicted/.
2020-10-28 06:26:11,070 [MainThread  ] [INFO ]  Processing file.90.txt.
2020-10-28 06:26:11,072 [MainThread  ] [INFO ]  Processing file.83.txt.
2020-10-28 06:26:11,073 [MainThread  ] [INFO ]  Processing file.14.txt.
2020-10-28 06:26:11,074 [MainThread  ] [INFO ]  Processing file.63.txt.
2020-10-28 06:26:11,076 [MainThread  ] [INFO ]  Processing file.34.txt.
2020-10-28 06:26:11,078 [MainThread  ] [INFO ]  Processing file.18.txt.
2020-10-28 06:26:11,080 [MainThread  ] [INFO ]  Processing file.55.txt.
2020-10-28 06:26:11,081 [MainThread  ] [INFO ]  Processing file.57.txt.


CalledProcessError: ignored

In [ ]:
!git clone https://github.com/tagucci/pythonrouge

Cloning into 'pythonrouge'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 314 (delta 0), reused 1 (delta 0), pack-reused 309
Receiving objects: 100% (314/314), 362.05 KiB | 18.10 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
!python pythonrouge/setup.py install

Traceback (most recent call last):
  File "pythonrouge/setup.py", line 31, in <module>
    long_description=open('README.md').read(),
FileNotFoundError: [Errno 2] No such file or directory: 'README.md'


In [ ]:
!pip3 install git+https://github.com/tagucci/pythonrouge.git


  Cloning https://github.com/tagucci/pythonrouge.git to /tmp/pip-req-build-9nip1y_b
  Running command git clone -q https://github.com/tagucci/pythonrouge.git /tmp/pip-req-build-9nip1y_b
  Created wheel for pythonrouge: filename=pythonrouge-0.2-cp36-none-any.whl size=285402 sha256=1e27719bb80512f75b9135843f17dd28c3050fc28e5e8cf9ac86f0fc3ab13070
  Stored in directory: /tmp/pip-ephem-wheel-cache-4p0yy6r3/wheels/fd/ff/be/6716935d513fa8656ab185cb0aa70aed382b72dda42bf09c95
Successfully built pythonrouge


In [ ]:
!ls "pythonrouge/pythonrouge/RELEASE-1.5.5"

data  README.txt  RELEASE-NOTE.txt  ROUGE-1.5.5.pl  runROUGE-test.pl  XML


In [ ]:
!bash "pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl" . exc WordNet-2.0.exc.db

pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl: line 2: use: command not found
pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl: line 3: @ARGV!=3: command not found
pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl: line 4: syntax error near unexpected token `DIR,$ARGV[0]'
pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl: line 4: `opendir(DIR,$ARGV[0])||die "Cannot open directory $ARGV[0]\n";'


In [ ]:
!ls "pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl"

pythonrouge/pythonrouge/RELEASE-1.5.5/data/WordNet-2.0-Exceptions/buildExeptionDB.pl
